# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [117]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [71]:
df = pd.read_csv('data/previsao_de_renda.csv')

In [72]:
df = df[list(df.columns)[3:]].dropna()
df.posse_de_veiculo = df.posse_de_veiculo.replace({True: 1, False: 0})
df.posse_de_imovel = df.posse_de_imovel.replace({True: 1, False: 0})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12427 non-null  object 
 1   posse_de_veiculo       12427 non-null  int64  
 2   posse_de_imovel        12427 non-null  int64  
 3   qtd_filhos             12427 non-null  int64  
 4   tipo_renda             12427 non-null  object 
 5   educacao               12427 non-null  object 
 6   estado_civil           12427 non-null  object 
 7   tipo_residencia        12427 non-null  object 
 8   idade                  12427 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  12427 non-null  float64
 11  renda                  12427 non-null  float64
dtypes: float64(3), int64(4), object(5)
memory usage: 1.2+ MB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [73]:
X = df.drop(columns=['renda'])
y = df['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [74]:
df_train = pd.concat([X_train, y_train], axis=1)

In [75]:
alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

model = 'renda ~ sexo +\
                 posse_de_veiculo +\
                 posse_de_imovel +\
                 qtd_filhos +\
                 tipo_renda +\
                 educacao +\
                 estado_civil +\
                 tipo_residencia +\
                 idade +\
                 tempo_emprego +\
                 qt_pessoas_residencia'

for a in alpha:

    md = smf.ols(model, df_train)
    md_fitted = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0.01, alpha = a)

    df_y_test = pd.concat([y_test, md_fitted.predict(X_test)], axis=1)
    r2_test = df_y_test.corr().iloc[1,0]**2
    
    print(f"Alpha {a}")
    print(f"R2 Train {round(md_fitted.rsquared,4)} / R2 Test {round(r2_test,4)}\n")

Alpha 0
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.001
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.005
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.01
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.05
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.1
R2 Train 0.2571 / R2 Test 0.2532



3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [76]:
for a in alpha:

    md = smf.ols(model, df_train)
    md_fitted = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = a)

    df_y_test = pd.concat([y_test, md_fitted.predict(X_test)], axis=1)
    r2_test = df_y_test.corr().iloc[1,0]**2
    
    print(f"Alpha {a}")
    print(f"R2 Train {round(md_fitted.rsquared,4)} / R2 Test {round(r2_test,4)}\n")

Alpha 0
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.001
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.005
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.01
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.05
R2 Train 0.2571 / R2 Test 0.2532

Alpha 0.1
R2 Train 0.2571 / R2 Test 0.2532



4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?

In [77]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


In [78]:
X_dummy = pd.get_dummies(X)
variaveis = stepwise_selection(X_dummy, np.asarray(y))

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 0.0
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Empresário          with p-value 1.75299e-07
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário']
Add  idade                          with p-value 1.9605e-07
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade']
Add  sexo_F                         with p-value 1.43093e-96
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'sexo_F']
Add  educacao_Superior completo     with p-value 3.07164e-06
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'sexo_F', 'educacao_Superior completo']
Add  estado_civil_Casado            with p-value 0.00727196
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'sexo_F', 'educacao_Superior completo', 'estado_civil_Casado']
Add  posse_de_imovel             

In [93]:
modelo = 'renda ~ tempo_emprego + sexo_M + tipo_renda_Empresário + idade + sexo_F + educacao_Superior_completo + estado_civil_Casado + posse_de_imovel'

df_train_dummies = pd.get_dummies(df_train)
df_train_dummies.columns = ['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda', 'sexo_F', 'sexo_M',
       'tipo_renda_Assalariado', 'tipo_renda_Bolsista',
       'tipo_renda_Empresário', 'tipo_renda_Pensionista',
       'tipo_renda_Servidor público', 'educacao_Primário',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior_completo', 'educacao_Superior_incompleto',
       'estado_civil_Casado', 'estado_civil_Separado', 'estado_civil_Solteiro',
       'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Aluguel',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental']

In [94]:
md = smf.ols(modelo, df_train_dummies)
md_fitted = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.01)

In [111]:
X_test_dummies = pd.get_dummies(X_test)
X_test_dummies.columns = ['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'sexo_F', 'sexo_M',
       'tipo_renda_Assalariado', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor público',
       'educacao_Primário', 'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior_completo', 'educacao_Superior_incompleto',
       'estado_civil_Casado', 'estado_civil_Separado', 'estado_civil_Solteiro',
       'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Aluguel',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental']

In [112]:
df_y_test = pd.concat([y_test, md_fitted.predict(X_test_dummies)], axis=1)
r2_test = df_y_test.corr().iloc[1,0]**2

print(f"R2 Train {round(md_fitted.rsquared,4)} / R2 Test {round(r2_test,4)}\n")

R2 Train 0.2564 / R2 Test 0.3092



5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

<div style="background: #EEEEF4; padding: 15px 20px; border-radius: 5px; margin-top: 20px">
    <p style="font-size: 11px; color: #666; font-weight: bold">Resposta</p>
    <p>O melhor modelo foi o último, onde o R2 de Teste foi de 0.31</p>
</div>

6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [116]:
modelo = 'np.log(renda) ~ tempo_emprego + sexo_M + tipo_renda_Empresário + idade + sexo_F + educacao_Superior_completo + estado_civil_Casado + posse_de_imovel'
md = smf.ols(modelo, df_train_dummies)
md_fitted = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.01)

df_y_test = pd.concat([y_test, np.exp(md_fitted.predict(X_test_dummies))], axis=1)
r2_test = df_y_test.corr().iloc[1,0]**2

print(f"R2 Train {round(md_fitted.rsquared,4)} / R2 Test {round(r2_test,4)}\n")

R2 Train 0.3515 / R2 Test 0.3773



<div style="background: #EEEEF4; padding: 15px 20px; border-radius: 5px; margin-top: 20px">
    <p style="font-size: 11px; color: #666; font-weight: bold">Resposta</p>
    <p>Transformando a variável renda com Log conseguimos aumentat o R2 de teste para 0.38</p>
</div>

7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [130]:
X_dummies = pd.get_dummies(X)

X_train_dummies, X_test_dummies, y_train, y_test = train_test_split(X_dummies, y, test_size=0.25)

tree = DecisionTreeRegressor(max_depth=10)

tree.fit(X_train_dummies, y_train)

r2_train = tree.score(X_train_dummies, y_train)
r2_test = tree.score(X_test_dummies, y_test)

print(f"O R² de Treino da árvore é: {r2_train}")
print(f"O R² de Teste da árvore é: {r2_test}")

O R² de Treino da árvore é: 0.6729192757261102
O R² de Teste da árvore é: 0.5149422961053773
